Extra processing steps required for pipelineclasses.py for the 2022.1 Release. Model was primarily built using the 2024.1 Release columns and values

In [1]:
import pandas as pd

df = pd.read_parquet('https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2022/resstock_tmy3_release_1/metadata_and_annual_results/national/parquet/upgrade01_metadata_and_annual_results.parquet')
df.head(500).to_parquet('testfiles/2022_1_first_500.parquet')

In [5]:
df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')
column_plan_df = pd.read_csv('../Training/column_plan.csv', usecols=['field_name','keep_for_model'])
# All the columns that we ultimately need for the model
needed_cols = column_plan_df.loc[
    (column_plan_df['keep_for_model'] == 'Yes') | 
    (column_plan_df['keep_for_model'] == 'Split')
]['field_name'].to_list()
cols_2022 = set([col for col in df_2022.columns if col.startswith('in')])
cols_2024 = set([col for col in df_2024.columns if col.startswith('in')])

print(f'2024 but not 2022: {(set(needed_cols) & cols_2024) - cols_2022}')
print(f'2022 but not 2024: {(set(needed_cols) & cols_2022) - cols_2024}')

2024 but not 2022: {'in.duct_leakage_and_insulation', 'in.household_has_tribal_persons', 'in.clothes_washer_usage_level', 'in.cooking_range_usage_level', 'in.duct_location', 'in.clothes_dryer_usage_level', 'in.refrigerator_usage_level'}
2022 but not 2024: set()


In [13]:
# in.clothes_wasger_usage_level

df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')

print('2022 before')
print(df_2022['in.clothes_washer'].unique())

df_2022['in.clothes_washer'] = df_2022['in.clothes_washer'].str.split(', ',expand=True)[0]
df_2022['in.clothes_washer_usage_level'] = df_2022['in.usage_level'].map({"Low" : "80% Usage", "Medium":"100% Usage", "High":"120% Usage"})

print('2022 after')
print(df_2022['in.clothes_washer'].unique())
print(df_2022['in.clothes_washer_usage_level'].unique())

print('2024')
print(df_2024['in.clothes_washer'].unique())
print(df_2024['in.clothes_washer_usage_level'].unique())

if set(df_2024['in.clothes_washer'].unique()) == set(df_2022['in.clothes_washer'].unique()):
    print("in.clothes_washer: PASS")
else:
    print("in.clothes_washer: FAIL")

if set(df_2024['in.clothes_washer_usage_level'].unique()) == set(df_2022['in.clothes_washer_usage_level'].unique()):
    print("in.clothes_washer_usage_level: PASS")
else:
    print("in.clothes_washer_usage_level: FAIL")

2022 before
['Standard, 80% Usage' 'EnergyStar, 100% Usage' 'EnergyStar, 80% Usage'
 'EnergyStar, 120% Usage' 'None' 'Standard, 100% Usage'
 'Standard, 120% Usage']
2022 after
['Standard' 'EnergyStar' 'None']
['80% Usage' '100% Usage' '120% Usage']
2024
['Standard' 'None' 'EnergyStar']
['100% Usage' '80% Usage' '120% Usage']
in.clothes_washer: PASS
in.clothes_washer_usage_level: PASS


In [14]:
# in.clothes_dryer_usage_level

df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')

print('2022 before')
print(df_2022['in.clothes_dryer'].unique())

df_2022['in.clothes_dryer'] = df_2022['in.clothes_dryer'].str.split(', ',expand=True)[0]
df_2022['in.clothes_dryer_usage_level'] = df_2022['in.usage_level'].map({"Low" : "80% Usage", "Medium":"100% Usage", "High":"120% Usage"})

print('2022 after')
print(df_2022['in.clothes_dryer'].unique())
print(df_2022['in.clothes_dryer_usage_level'].unique())

print('2024')
print(df_2024['in.clothes_dryer'].unique())
print(df_2024['in.clothes_dryer_usage_level'].unique())

if set(df_2024['in.clothes_dryer'].unique()) == set(df_2022['in.clothes_dryer'].unique()):
    print("in.clothes_dryer: PASS")
else:
    print("in.clothes_dryer: FAIL")

if set(df_2024['in.clothes_dryer_usage_level'].unique()) == set(df_2022['in.clothes_dryer_usage_level'].unique()):
    print("in.clothes_dryer_usage_level: PASS")
else:
    print("in.clothes_dryer_usage_level: FAIL")

2022 before
['Electric, 80% Usage' 'Electric, 100% Usage' 'Electric, 120% Usage'
 'None' 'Gas, 100% Usage' 'Gas, 80% Usage' 'Gas, 120% Usage'
 'Propane, 120% Usage']
2022 after
['Electric' 'None' 'Gas' 'Propane']
['80% Usage' '100% Usage' '120% Usage']
2024
['Electric' 'None' 'Gas' 'Propane']
['100% Usage' '80% Usage' '120% Usage']
in.clothes_dryer: PASS
in.clothes_dryer_usage_level: PASS


In [15]:
# in.cooking_range_usage_level

df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')

print('2022 before')
print(df_2022['in.cooking_range'].unique())

df_2022['in.cooking_range'] = df_2022['in.cooking_range'].str.split(', ',expand=True)[0]
# 2022 model does not have support for differentiation between Induction and Electric Resistance, so simply assume everyone has Electric Resistance - I am assuming that is how the simulation was run. Website will not be able to capture any differences on 2022 measures on this feature
df_2022['in.cooking_range'] = df_2022['in.cooking_range'].map({"Electric" : "Electric Resistance"})

df_2022['in.cooking_range_usage_level'] = df_2022['in.usage_level'].map({"Low" : "80% Usage", "Medium":"100% Usage", "High":"120% Usage"})

print('2022 after')
print(df_2022['in.cooking_range'].unique())
print(df_2022['in.cooking_range_usage_level'].unique())

print('2024')
print(df_2024['in.cooking_range'].unique())
print(df_2024['in.cooking_range_usage_level'].unique())

if set(df_2024['in.cooking_range'].unique()) == set(df_2022['in.cooking_range'].unique()):
    print("in.cooking_range: PASS")
else:
    print("in.cooking_range: FAIL")

if set(df_2024['in.cooking_range_usage_level'].unique()) == set(df_2022['in.cooking_range_usage_level'].unique()):
    print("in.cooking_range_usage_level: PASS")
else:
    print("in.cooking_range_usage_level: FAIL")

2022 before
['Electric, 80% Usage' 'Electric, 100% Usage' 'Electric, 120% Usage'
 'Gas, 80% Usage' 'Gas, 100% Usage' 'Gas, 120% Usage' 'None'
 'Propane, 120% Usage' 'Propane, 100% Usage' 'Propane, 80% Usage']
2022 after
['Electric' 'Gas' 'None' 'Propane']
['80% Usage' '100% Usage' '120% Usage']
2024
['Electric Resistance' 'Gas' 'Propane' 'Electric Induction' 'None']
['100% Usage' '80% Usage' '120% Usage']
in.cooking_range: FAIL
in.cooking_range_usage_level: PASS


In [17]:
# in.refrigerator_usage_level

df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')

print('2022 before')
print(df_2022['in.refrigerator'].unique())

# 2024 also has an EF 21.9 option - should not affect model
df_2022['in.refrigerator'] = df_2022['in.refrigerator'].str.split(', ',expand=True)[0]
df_2022['in.refrigerator_usage_level'] = df_2022['in.usage_level'].map({"Low" : "95% Usage", "Medium":"100% Usage", "High":"105% Usage"})

print('2022 after')
print(df_2022['in.refrigerator'].unique())
print(df_2022['in.refrigerator_usage_level'].unique())

print('2024')
print(df_2024['in.refrigerator'].unique())
print(df_2024['in.refrigerator_usage_level'].unique())

if set(df_2024['in.refrigerator'].unique()) == set(df_2022['in.refrigerator'].unique()):
    print("in.refrigerator: PASS")
else:
    print("in.refrigerator: FAIL")

if set(df_2024['in.refrigerator_usage_level'].unique()) == set(df_2022['in.refrigerator_usage_level'].unique()):
    print("in.refrigerator_usage_level: PASS")
else:
    print("in.refrigerator_usage_level: FAIL")

2022 before
['EF 17.6, 100% Usage' 'EF 15.9, 100% Usage' 'EF 10.5, 100% Usage'
 'EF 6.7, 100% Usage' 'EF 19.9, 100% Usage' 'None' 'EF 10.2, 100% Usage']
2022 after
['EF 17.6' 'EF 15.9' 'EF 10.5' 'EF 6.7' 'EF 19.9' 'None' 'EF 10.2']
['95% Usage' '100% Usage' '105% Usage']
2024
['EF 19.9' 'EF 21.9' 'EF 17.6' 'EF 15.9' 'EF 6.7' 'EF 10.5' 'None'
 'EF 10.2']
['100% Usage' '95% Usage' '105% Usage']
in.refrigerator: FAIL
in.refrigerator_usage_level: PASS


In [19]:
# in.duct_leakage_and_insulation

df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')

print('2022')
print(df_2022['in.ducts'].unique())
print(df_2024['in.duct_leakage_and_insulation'].unique())

# 2024 also has an EF 21.9 option - should not affect model
df_2022 = df_2022.rename(columns={'in.ducts':"in.duct_leakage_and_insulation"})


if set(df_2024['in.duct_leakage_and_insulation'].unique()) == set(df_2022['in.duct_leakage_and_insulation'].unique()):
    print("in.duct_leakage_and_insulation: PASS")
else:
    print("in.duct_leakage_and_insulation: FAIL")

2022
['30% Leakage, R-8' '0% Leakage, Uninsulated' '20% Leakage, R-4'
 '20% Leakage, R-8' 'None' '10% Leakage, R-4' '20% Leakage, Uninsulated'
 '30% Leakage, R-4' '10% Leakage, Uninsulated' '30% Leakage, Uninsulated'
 '10% Leakage, R-8' '20% Leakage, R-6' '10% Leakage, R-6'
 '30% Leakage, R-6']
['20% Leakage, Uninsulated' '30% Leakage, R-4' '0% Leakage, Uninsulated'
 '20% Leakage, R-4' '10% Leakage, Uninsulated' '20% Leakage, R-6' 'None'
 '20% Leakage, R-8' '10% Leakage, R-4' '30% Leakage, Uninsulated'
 '10% Leakage, R-8' '30% Leakage, R-6' '30% Leakage, R-8'
 '10% Leakage, R-6']
in.duct_leakage_and_insulation: PASS


In [ ]:
# in.duct_location and in.household_has_tribal_persons

# This information is not included in 2022 - unlikely to use it in the website so it should not matter too much

In [3]:
# import sys, os
import pandas as pd
# project_dir = os.path.abspath('..')
# print(project_dir)
# sys.path.append(project_dir)
df_2022 = pd.read_parquet('testfiles/2022_1_first_500.parquet')
df_2024 = pd.read_parquet('testfiles/2024_1_first_500.parquet')

from pipelineclasses import Preprocessing

df_2022_processed = Preprocessing("2022_1").transform(df_2022)
df_2024_processed = Preprocessing("2024_1").transform(df_2024)

cols_2022 = set(df_2022_processed.columns.to_list())
cols_2024 = set(df_2024_processed.columns.to_list())

print(f' Columns in 2022 but not 2024: {cols_2022 - cols_2024}')
print(f' Columns in 2024 but not 2022: {cols_2024 - cols_2022}')

for col in df_2022_processed.columns:
    uniq_2024 = set(df_2024_processed[col].unique())
    uniq_2022 = set(df_2022_processed[col].unique())
    if uniq_2022 != uniq_2024:
        print(col)
        print(f'Values in 2022 but not 2024: {uniq_2022 - uniq_2024}')
        print(f'Values in 2024 but not 2022: {uniq_2024 - uniq_2022}')


 Columns in 2022 but not 2024: set()
 Columns in 2024 but not 2022: {'in.duct_location', 'in.household_has_tribal_persons'}
in.cooking_range
Values in 2022 but not 2024: {nan}
Values in 2024 but not 2022: {'None', 'Electric Induction', 'Propane', 'Gas'}
in.cooling_setpoint_offset_period
Values in 2022 but not 2024: {'Day Setup -2h', 'Day Setup -5h', 'Day and Night Setup +5h', 'Day Setup and Night Setback +4h', 'Day Setup and Night Setback -2h', 'Day Setup and Night Setback +2h', 'Day Setup and Night Setback -5h', 'Day Setup and Night Setback -1h', 'Day and Night Setup +2h', 'Day Setup +2h', 'Day Setup +3h', 'Day Setup +1h', 'Day Setup -1h', 'Day and Night Setup +1h', 'Day Setup and Night Setback +3h', 'Day Setup and Night Setback -4h', 'Day and Night Setup -3h'}
Values in 2024 but not 2022: {'Day Setup and Night Setback +5h', 'Day and Night Setup -5h'}
in.county
Values in 2022 but not 2024: {'G3600150', 'G3900670', 'G2100150', 'G3600910', 'G3400030', 'G1701110', 'G2201210', 'G2601290',